In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
seed = 30
tf.random.set_seed(seed)
np.random.seed(seed)

# Load model and test it locally

In [2]:
def get_data(file_path):
    audio_binary = tf.io.read_file(file_path)
    audio, _ = tf.audio.decode_wav(audio_binary)
    waveform = tf.squeeze(audio, axis=-1)    
    
    zero_padding = tf.zeros([10000000] - tf.shape(waveform), dtype=tf.float32)
    waveform = tf.cast(waveform, tf.float32)
    equal_length = tf.concat([waveform, zero_padding], 0)
    
    spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, -1)
    return spectrogram

In [3]:
new_model = tf.keras.models.load_model('saved_model/base_line/20210210')
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing_2 (Resizing)        (None, 32, 32, 1)         0         
_________________________________________________________________
normalization_2 (Normalizati (None, 32, 32, 1)         3         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 16)        160       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [4]:
file = '/tf/Berkeley/W210/dementia/0extra/ADReSS-IS2020-data 2/train/Full_wave_enhanced_audio/cc/S043.wav'
new_model.predict(tf.expand_dims(get_data(file), axis = 0), verbose=2)

1/1 - 0s


array([[28.210304]], dtype=float32)

# Query the server

In [5]:
import json
import requests

In [6]:
file = '/tf/Berkeley/W210/dementia/0extra/ADReSS-IS2020-data 2/train/Full_wave_enhanced_audio/cc/S043.wav'

spectrogram = tf.expand_dims(get_data(file), axis = 0).numpy().tolist()

In [7]:
data = json.dumps({ 
    "instances": spectrogram
})
headers = {"content-type": "application/json"}

In [8]:
response = requests.post('http://model_server:8501/v1/models/base_line:predict', data=data, headers=headers)
result = response.json()['predictions'][0][0]
print(result)

28.2103043


In [9]:
# written to model_serving_request.py

def model_serving_request(filepath, server_ip):

    import tensorflow as tf
    import numpy as np
    import json
    import requests	

    audio_binary = tf.io.read_file(filepath)
    audio, _ = tf.audio.decode_wav(audio_binary)

    waveform = tf.squeeze(audio, axis=-1)    
    zero_padding = tf.zeros([10000000] - tf.shape(waveform), dtype=tf.float32)
    waveform = tf.cast(waveform, tf.float32)
    equal_length = tf.concat([waveform, zero_padding], 0)
    
    spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, -1)

    spectrogram = tf.expand_dims(get_data(filepath), axis = 0).numpy().tolist()
    data = json.dumps({
        "instances": spectrogram
        })
    headers = {"content-type": "application/json"}
    response = requests.post('http://' + server_ip + ':8501/v1/models/base_line:predict', data=data, headers=headers)
    result = response.json()['predictions'][0][0]

    return result


In [10]:
model_serving_request(filepath = file, server_ip = 'model_server')

28.2103043